<a href="https://colab.research.google.com/github/ahmad123576/Deep-Learning/blob/main/pipeline_prefetch%26cache.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# optimize tensorflow pipeline performance with prefetch and cache

In [2]:
import tensorflow as tf
import time

In [3]:
tf.__version__

'2.18.0'

In [4]:
# prefetch
class Filedataset(tf.data.Dataset):
    def read_files(self):
      # oprning the file
        time.sleep(0.03)

        for sample_idx in range(10):
          # reading data(line,record) from the file
            yield (sample_idx,)

    def __new__(cls, num_samples=3):
      return tf.data.Dataset.from_generator(
          cls.read_files,
          output_signature=tf.TensorSpec(shape=(1,), dtype=tf.int64),
          args=(num_samples,)
      )


In [5]:
def benchmark(dataset, num_epochs=2):
  for epoch in range(num_epochs):
    for sample in dataset:
      # performing a training step
      time.sleep(0.01)

In [6]:
%%timeit
benchmark(Filedataset())

466 ms ± 85.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
benchmark(Filedataset().prefetch(1))

353 ms ± 6.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
benchmark(Filedataset().prefetch(tf.data.AUTOTUNE))


508 ms ± 143 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
# as you noticed that prefetch improves our performance from 468ms to 389 and 388

In [10]:
# cache

In [13]:
dataset=tf.data.Dataset.range(5)
dataset=dataset.map(lambda x:x**2)
dataset=dataset.cache("mycache.txt")
list(dataset.as_numpy_iterator())

[np.int64(0), np.int64(1), np.int64(4), np.int64(9), np.int64(16)]

In [14]:
def mapped_function(s):
  # do some hard pre-processing
  tf.py_function(lambda: time.sleep(0.03), [], ())
  return s

In [16]:
%%timeit
benchmark(Filedataset().map(mapped_function),5)

2.02 s ± 120 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
benchmark(Filedataset().map(mapped_function).cache(),5)

874 ms ± 56.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
# performance improved